In [1]:
# -------------------------------------------------------
# Project1
# Written by Yilin Li(40083064), Yuhua Jiang(40083453)
# For COMP 6721 Section (lab FJ,FI) – Fall 2019
# --------------------------------------------------------
import math
import time
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import shapefile
from matplotlib.lines import Line2D

# set timmer
time1 = 0.000
time2 = 0.000

# set the original map
x1 = -73.590
x2 = -73.550
y1 = 45.490
y2 = 45.530

# gridSize and boundary adjustment
gridSize = round(float(input("Please input the grid size:")), 3)
if(round((x2-x1),3)%gridSize!=0.0):
    x2=round((x1+round((math.ceil(round((x2-x1),3)/gridSize)*gridSize),3)),3)
    y2=round((y1+round((math.ceil(round((y2-y1),3)/gridSize)*gridSize),3)),3)
# print("x1,x2 ",x1,x2)
# print("y1,y2 ",y1,y2)

sizeX = int(round((x2-x1),3) / gridSize)
sizeY = int(round((y2-y1),3)/ gridSize)

# set start and end
startX = 0.000
startY = 0.000
endX = 0.000
endY = 0.000

startX = -73.55
startY = 45.49
endX = -73.59
endY = 45.53

# set distribution
totalAll = 0.000
mean = 0.000
average = 0.000
standardDeviation = 0.00
threshold = 0
sortTotal = []


# X Y is the smallest boundary of an object in each grid
class Grid:
    axisX = 0.000
    axisY = 0.000
    total = 0
    crime = False

# initial the grid map
myGrid = [[Grid() for col in range(sizeX)] for row in range(sizeY)]
for i in range(len(myGrid)):
    for j in range(len(myGrid[0])):
        myGrid[i][j] = Grid()
        myGrid[i][j].axisX = round(x1 + gridSize * int(i), 3)
        myGrid[i][j].axisY = round(y1 + gridSize * int(j), 3)

# read crime file
sf = shapefile.Reader(r"crime_dt.shp")
shapes = sf.shapes()

for i in range(len(sf)):
    shape = sf.shape(i)
    shapeTem = shape.points[0]
    shapeX = shapeTem[0]
    shapeY = shapeTem[1]
    # set the boundary
    if shapeX < x1 or shapeX > x2 or shapeY < y1 or shapeY > y2:
        break
    grid_X = (shapeX - x1) // gridSize
    grid_Y = (shapeY - y1) // gridSize
    # put point in the grid (initial total in each grid)
    myGrid[int(grid_X)][int(grid_Y)].total = myGrid[int(grid_X)][int(grid_Y)].total + 1

# calculate the total point
for i in range(len(myGrid)):
    for j in range(len(myGrid[i])):
        totalAll = totalAll + myGrid[i][j].total
        sortTotal.append(myGrid[i][j].total)
sortTotal.sort()

threshold = input("Please input threshold:")
index = int(int(threshold) / 100 * len(sortTotal))  # threshold begin at this index

# output calculated data
print("Total: ", totalAll)
sizeTem = len(sortTotal)
medium = (sortTotal[sizeTem // 2] + sortTotal[sizeTem // 2 - 1]) / 2
print("Medium: ", medium)
average = totalAll / sizeTem
print("Average: ", average)
totalTem = 0.00
for i in sortTotal:
    totalTem = (i - average) * (i - average)
standardDeviation = math.sqrt(totalTem / sizeTem)
print("Standard deviation: ", standardDeviation)
print()


# show the map
fig, axes = plt.subplots(1, 2, figsize=(13, 6))
axisX=[]
axisY=[]
temX=round(x1,3)
temY=round(y1,3)
for i in range(sizeX+1):
    axisX.append(temX)
    temX=round((temX+gridSize),3)
for i in range(sizeY+1):
    axisY.append(temY)
    temY=round((temY+gridSize),3)

for ax in axes:
    ax.plot()
    ax.axis([x1, x2, y1, y2])
    ax.set_xticks(axisX)
    ax.set_yticks(axisY)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)

axes[0].set_title("Best-first Searching")
axes[0].grid()
axes[0].set_facecolor("darkblue")
axes[1].set_title("A* Searching")
axes[1].grid()
axes[1].set_facecolor("darkblue")

# show the criminal block
rectlist = []
for i in range(len(myGrid)):
    for j in range(len(myGrid[i])):
        if myGrid[i][j].total >= sortTotal[index]:
            myGrid[i][j].crime = True
            axes[0].add_patch(
                patches.Rectangle((myGrid[i][j].axisX, myGrid[i][j].axisY), gridSize, gridSize, color='yellow'))
            axes[1].add_patch(
                patches.Rectangle((myGrid[i][j].axisX, myGrid[i][j].axisY), gridSize, gridSize, color='yellow'))

# calculate the average crime rate in the range between the current grid and goal grid
def hAverage(i1, j1, i2, j2):
    average = 0.00
    totalCount = 0.00
    for i in range(i1, i2 + 1):
        for j in range(j1, j2 + 1):
            totalCount = totalCount + myGrid[i][j].total
    if (i1 == i2 and j1 == j2):
        average = round(totalCount, 3)
    if (i1 == i2 and j1 != j2):
        average = round((totalCount / (j2 - j1)), 3)
    if (j1 == j2 and i1 != i2):
        average = round((totalCount / (i2 - i1)), 3)
    if (j1 != j2 and i1 != i2):
        average = round((totalCount / ((i2 - i1) * (j2 - j1))), 3)
    return average

# according XY find ij of each grid
def findGrid(x, y):
    for i in range(len(myGrid)):
        for j in range(len(myGrid[i])):
            if (round((x - myGrid[i][j].axisX), 3) < gridSize and round((y - myGrid[i][j].axisY), 3) < gridSize):
                return int(i), int(j)
            if (round((x - myGrid[i][j].axisX), 3) <= gridSize and round((y - myGrid[i][j].axisY),
                                                                         3) < gridSize and x == x2):
                return int(i), int(j)
            if (round((x - myGrid[i][j].axisX), 3) < gridSize and round((y - myGrid[i][j].axisY),
                                                                        3) <= gridSize and y == y2):
                return int(i), int(j)
    return 0, 0

# according XY find ij of each node
def findNode(x, y):
    i = math.floor(round((round(abs(x - x1),3) / gridSize),3))
    j = math.floor(round((round(abs(y - y1),3) / gridSize),3))
    return i, j

# put the start point and end point on the graph
def findStartAndEnd(x, y):
    if (nodeInMap(x, y)):
        i, j = findNode(x, y)
        myX = round((x1 + gridSize * i), 3)
        myY = round((y1 + gridSize * j), 3)
        return myX, myY

# judge if the point in the map
def nodeInMap(x, y):
    if (x < x1 or x > x2 or y < y1 or y > y2):
        return False
    else:
        return True

# judge if the grid is existed
def gridInMap(i, j):
    if (i < sizeX and i >= 0 and j < sizeY and j >= 0):
        return True
    else:
        return False

# judge if the neighbours of the current node is existed
def findNeighInMap(id, i, j):
    pointX = round((x1 + i * gridSize), 3)
    pointY = round((y1 + j * gridSize), 3)
    neighX = pointX
    neighY = pointY
    if (id == 0):
        neighX = pointX
        neighY = pointY + gridSize
    if (id == 1):
        neighX = pointX + gridSize
        neighY = pointY + gridSize
    if (id == 2):
        neighX = pointX + gridSize
        neighY = pointY
    if (id == 3):
        neighX = pointX + gridSize
        neighY = pointY - gridSize
    if (id == 4):
        neighX = pointX
        neighY = pointY - gridSize
    if (id == 5):
        neighX = pointX - gridSize
        neighY = pointY - gridSize
    if (id == 6):
        neighX = pointX - gridSize
        neighY = pointY
    if (id == 7):
        neighX = pointX - gridSize
        neighY = pointY + gridSize

    if (nodeInMap(round(neighX, 3), round(neighY, 3))):
        return True
    else:
        return False

def findNeigh(id, i, j):
    neighI = i
    neighJ = j
    if (id == 0):
        neighJ = neighJ + 1
    if (id == 1):
        neighI = neighI + 1
        neighJ = neighJ + 1
    if (id == 2):
        neighI = neighI + 1
    if (id == 3):
        neighI = neighI + 1
        neighJ = neighJ - 1
    if (id == 4):
        neighJ = neighJ - 1
    if (id == 5):
        neighI = neighI - 1
        neighJ = neighJ - 1
    if (id == 6):
        neighI = neighI - 1
    if (id == 7):
        neighI = neighI - 1
        neighJ = neighJ + 1
    return neighI, neighJ

# neighbor cost
def neighCostCase(id, i, j):
    cost = 0.00
    if (id == 0):  # the upper node
        if (gridInMap(i - 1, j)):  # first left grid is in the map
            if (gridInMap(i, j)):  # first right grid is in the map
                if (myGrid[i - 1][j].crime == False and myGrid[i][j].crime == False):
                    cost = 1.0
                elif ((myGrid[i - 1][j].crime == True and myGrid[i][j].crime == False) or (
                        myGrid[i - 1][j].crime == False and myGrid[i][j].crime == True)):
                    cost = 1.3
                else:
                    cost = 99999.0
            else:  # current node is on the right boundary of the map
                cost = 99999.0
        else:  # current node is on the left boundary of the map
            cost= 99999.0
        return cost
    if (id == 1):  # node at first right conner
        if (myGrid[i][j].crime == False):
            cost = 1.50
        else:
            cost = 99999.0
        return cost
    if (id == 2):  # node at right
        if (gridInMap(i, j)):  # first right grid is in the map
            if (gridInMap(i, j - 1)):  # second right grid is in the map
                if (myGrid[i][j].crime == False and myGrid[i][j - 1].crime == False):
                    cost = 1.0
                elif ((myGrid[i][j].crime == True and myGrid[i][j - 1].crime == False) or (
                        myGrid[i][j].crime == False and myGrid[i][j - 1].crime == True)):
                    cost = 1.3
                else:
                    cost = 99999.0
            else:  # current node is on the bottom boundary
                cost= 99999.0
        else:  # current node is on the top boundary
            cost = 99999.0
        return cost
    if (id == 3):  # the node at the second right conner
        if (myGrid[i][j - 1].crime == False):
            cost = 1.50
        else:
            cost = 99999.0
        return cost
    if (id == 4):  # the node at the bottom
        if (gridInMap(i - 1, j - 1)):  # second left grid is in the map
            if (gridInMap(i, j - 1)):  # second right grid is in the map
                if (myGrid[i - 1][j - 1].crime == False and myGrid[i][j - 1].crime == False):
                    cost = 1.0
                elif ((myGrid[i - 1][j - 1].crime == True and myGrid[i][j - 1].crime == False) or (
                        myGrid[i - 1][j - 1].crime == False and myGrid[i][j - 1].crime == True)):
                    cost = 1.3
                else:
                    cost = 99999.0
            else:  # current node is on the right boundary
                cost = 99999.0
        else:  # current node is on the left boundary
            cost = 99999.0
        return cost
    if (id == 5):  # node at the second left conner
        if (myGrid[i - 1][j - 1].crime == False):
            cost = 1.50
        else:
            cost = 99999.0
        return cost
    if (id == 6):  # node at left
        if (gridInMap(i - 1, j)):  # first left grid is in the map
            if (gridInMap(i - 1, j - 1)):  # second left grid is in the map
                if (myGrid[i - 1][j].crime == False and myGrid[i - 1][j - 1].crime == False):
                    cost = 1.0
                elif ((myGrid[i - 1][j].crime == True and myGrid[i - 1][j - 1].crime == False) or (
                        myGrid[i - 1][j].crime == False and myGrid[i - 1][j - 1].crime == True)):
                    cost = 1.3
                else:
                    cost = 99999.0
            else:  # the current node is on the bottom boundary
                cost = 99999.0
        else:  # the current node is on the top boundary
            cost = 99999.0
        return cost
    if (id == 7):  # the note at the first left conner
        if (myGrid[i - 1][j].crime == False):
            cost = 1.50
        else:
            cost = 99999.0
        return cost

    return cost


# Build a graph to store information of node
class Node:
    coordinateX = 0.00
    coordinateY = 0.00
    a = 0.00  # average criminal rate
    d = 0.00  # diagonal distance
    h = 0.00  # heuristic cost  h = (a/(totalAll/(sizeX*sizeY)))*d
    g = 0.00  # past cost
    f = 0.00  # h + g
    neighbourCost = []
    visited = False

    def __str__(self) -> str:
        return '(%s,%s)' % (self.coordinateX, self.coordinateY)

    def __repr__(self) -> str:
        return str(self)


def initialGraph(myGraph):
    for i in range(len(myGraph)):
        for j in range(len(myGraph[0])):

            myGraph[i][j] = Node()
            myGraph[i][j].coordinateX = round((x1 + i * gridSize), 3)
            myGraph[i][j].coordinateY = round((y1 + j * gridSize), 3)

            # diagonal distance
            dx = (round(abs(endX - myGraph[i][j].coordinateX), 3)) / gridSize
            dy = (round(abs(endY - myGraph[i][j].coordinateY), 3)) / gridSize
            myGraph[i][j].d = round(((dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)), 3)

            # the average criminal rate in the current node and the goal point area
            aveC = 0.00
            endI, endJ = findGrid(endX, endY)

            if (endX >= myGraph[i][j].coordinateX and endY >= myGraph[i][
                j].coordinateY):  # the goal point is on upper right
                if (myGraph[i][j].coordinateX == x2):
                    aveC = hAverage(i - 1, j, endI, endJ)
                elif (myGraph[i][j].coordinateY == y2):
                    aveC = hAverage(i, j - 1, endI, endJ)
                else:
                    aveC = hAverage(i, j, endI, endJ)
            elif (endX > myGraph[i][j].coordinateX and endY < myGraph[i][j].coordinateY):  # the goal is on the lower right
                if (myGraph[i][j].coordinateX == x2):
                    aveC = hAverage(endI, endJ, i - 1, j - 1)
                elif (myGraph[i][j].coordinateY == y1):
                    aveC = hAverage(i, j, endI, endJ)
                else:
                    aveC = hAverage(i, endJ, endI, j - 1)
            elif (endX < myGraph[i][j].coordinateX and endY > myGraph[i][j].coordinateY):  # the goal is on the upper left
                if (myGraph[i][j].coordinateX == x1):
                    aveC = hAverage(i, j, endI, endJ)
                elif (myGraph[i][j].coordinateY == y2):
                    aveC = hAverage(endI, endJ, i - 1, j - 1)
                else:
                    aveC = hAverage(endI, j, i - 1, endJ)
            elif (endX <= myGraph[i][j].coordinateX and endY <= myGraph[i][j].coordinateY):  # the goal is on the lower left
                if (myGraph[i][j].coordinateX == x1):
                    aveC = hAverage(endI, endJ, i, j - 1)
                elif (myGraph[i][j].coordinateY == y1):
                    aveC = hAverage(endI, endJ, i - 1, j)
                else:
                    aveC = hAverage(endI, endJ, i - 1, j - 1)

            myGraph[i][j].a = aveC
            # h = (a / (totalAll / (sizeX * sizeY))) + 0.7 * d
            myGraph[i][j].h = round(((myGraph[i][j].a / (totalAll / (sizeX * sizeY))) + 0.70 * myGraph[i][j].d), 3)

            myList = []
            for id in range(8):
                curcost = 0.00
                if (findNeighInMap(id, i, j)):
                    curcost = neighCostCase(id, i, j)
                else:
                    curcost = 99999.0
                myList.append(curcost)

            myGraph[i][j].neighbourCost = myList

def findpath(current, closeList, numberAlgorithm):
    pointer1 = current
    costD = 0.00
    costC = 0.00
    while (True):

        pointer0 = closeList.get(pointer1)
        line = [(pointer0.coordinateX, pointer0.coordinateY), (pointer1.coordinateX, pointer1.coordinateY)]
        (line_xs, line_ys) = zip(*line)
        axes[numberAlgorithm].add_line(Line2D(line_xs, line_ys, linewidth=3, color='red'))

        if(pointer0.coordinateX==pointer1.coordinateX):
            costD = costD + 1
            if(pointer0.coordinateY>pointer1.coordinateY):
                gridI,gridJ=findGrid(pointer1.coordinateX,pointer1.coordinateY)
            else:
                gridI,gridJ=findGrid(pointer0.coordinateX,pointer0.coordinateY)
            if(myGrid[gridI][gridJ].crime==False and myGrid[gridI-1][gridJ].crime==False):
                costC=costC+1.0
            else:
                costC=costC+1.30
        elif(pointer0.coordinateY==pointer1.coordinateY):
            costD = costD + 1
            if(pointer0.coordinateY==pointer1.coordinateY and pointer0.coordinateX<pointer1.coordinateX):
                gridI, gridJ = findGrid(pointer0.coordinateX, pointer0.coordinateY)
            else:
                gridI, gridJ = findGrid(pointer1.coordinateX, pointer1.coordinateY)
            if(myGrid[gridI][gridJ].crime==False and myGrid[gridI][gridJ-1].crime==False):
                costC = costC + 1.0
            else:
                costC = costC + 1.30
        else:
            costD = costD + math.sqrt(2)
            costC = costC + 1.50

        pointer1 = pointer0

        if (pointer1.coordinateX == startX and pointer1.coordinateY == startY):
            if(numberAlgorithm==0):
                print("BestFirst path distance: ",costD)
                print("BestFirst path crime cost: ", round(costC,3))
            else:
                print("A* path distance: ", costD)
                print("A* path crime cost: ", round(costC,3))
            break


def bestFirst():
    openList = {}
    closeList = {}

    eflag = 0  # check if find the endpoint
    starti, startj = findNode(startX, startY)
    openList[myGraph[starti][startj]] = Node()

    while (len(openList) != 0):
        minH = 999999.0
        current = None
        for k, v in openList.items():
            if (k.h < minH):    # find the point with minimum h
                minH = round(k.h, 3)
                current = k
        current.visited = True
        closeList[current] = openList.get(current)  # add it to closeList
        openList.pop(current)

        if (current.coordinateX == endX and current.coordinateY == endY):   # found the endpoint
            eflag = 1
            if (time1 < 10):    # less than 10s
                findpath(current, closeList, 0)     # draw the path
            else:
                print("Time is up. The optimal path is not found.")
            break
        else:
            for i in range(8):  # neighbours
                if (current.neighbourCost[i] < 99999.0):    # not block
                    currentI, currentJ = findNode(current.coordinateX, current.coordinateY)
                    neighbourI, neighbourJ = findNeigh(i, currentI, currentJ)
                    if (myGraph[neighbourI][neighbourJ].visited == False):  # not visited before
                        if (not (myGraph[neighbourI][neighbourJ] in openList.keys())):  # not in openList
                            openList[myGraph[neighbourI][neighbourJ]] = current
    if (eflag == 0):    # not found path
        print("Due to blocks, no path is found. Please change the map and try again")

def aStar():
    openList = {}
    closeList = {}

    eflag = 0  # check if find the endpoint
    starti, startj = findNode(startX, startY)
    openList[myGraphN[starti][startj]] = Node()

    while (len(openList) != 0):
        minF = 99999999.0
        current = None
        for k, v in openList.items():
            if (k.f < minF):
                minF = round(k.f, 3)    # find the point with minimum f
                current = k
        current.visited = True
        closeList[current] = openList.get(current)  # add to closeList
        openList.pop(current)

        if (current.coordinateX == endX and current.coordinateY == endY):   # found the goal
            eflag = 1
            if (time2 < 10):
                findpath(current, closeList, 1)
            else:
                print("Time is up. The optimal path is not found.")
            break
        else:
            for i in range(8):  # neighbours
                if (current.neighbourCost[i] < 99999.0):    # not block
                    currentI, currentJ = findNode(current.coordinateX, current.coordinateY)
                    neighbourI, neighbourJ = findNeigh(i, currentI, currentJ)
                    if (myGraphN[neighbourI][neighbourJ].visited == False): # not visited before
                        temG = current.g + current.neighbourCost[i]
                        temF = temG + myGraphN[neighbourI][neighbourJ].h
                        if ((myGraphN[neighbourI][neighbourJ] in openList.keys())): # in openList
                            if(myGraphN[neighbourI][neighbourJ].g>temG):    # update its g and f
                                myGraphN[neighbourI][neighbourJ].g = temG
                                myGraphN[neighbourI][neighbourJ].f = temF
                        else:   # not in openList
                            myGraphN[neighbourI][neighbourJ].g= temG
                            myGraphN[neighbourI][neighbourJ].f = temF
                            openList[myGraphN[neighbourI][neighbourJ]] = current

    if (eflag == 0):    # not found path
        print("Due to blocks, no path is found. Please change the map and try again")


startX, startY = findStartAndEnd(startX, startY)    #reset start point and endpoint
endX, endY = findStartAndEnd(endX, endY)
myGraph = [[Node() for col in range(sizeX + 1)] for row in range(sizeY + 1)]
time_start1 = time.time()
initialGraph(myGraph)
bestFirst()
time_end1 = time.time()

myGraphN = [[Node() for col in range(sizeX + 1)] for row in range(sizeY + 1)]
time_start2 = time.time()
initialGraph(myGraphN)
aStar()
time_end2 = time.time()

time1 = round((time_end1 - time_start1), 3)
time2 = round((time_end2 - time_start2), 3)
print('time cost for "best-first search" :', time1, 's')
print('time cost for "A* search" :', time2, 's')

plt.show()

Please input the grid size:0.002
Please input threshold:75
Total:  19010.0
Medium:  36.5
Average:  47.525
Standard deviation:  11.52375

BestFirst path distance:  32.87005768508882
BestFirst path crime cost:  36.0
A* path distance:  30.627416997969533
A* path crime cost:  33.5
time cost for "best-first search" : 0.09 s
time cost for "A* search" : 0.101 s


<Figure size 1300x600 with 2 Axes>